In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Pool

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
from eth_defi.token import TokenDetails, fetch_erc20_details

In [9]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    args  = Conversion().decode_data(evt["data"])
    token0_address = Conversion().convert_uint256_hex_string_to_address(topics[1])
    token1_address = Conversion().convert_uint256_hex_string_to_address(topics[2])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = token0_address
    event['details']['token1'] = token1_address    
    if(len(args) == 5):
        event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(args[0], signed=direction(args[0])) 
        event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(args[1], signed=direction(args[1])) 
        event['details']['sqrt_price_x96'] = Conversion().convert_int256_bytes_to_int(args[2]) 
        event['details']['liquidity'] = Conversion().convert_int256_bytes_to_int(args[3]) 
        event['details']['tick'] = Conversion().convert_int256_bytes_to_int(args[4], signed=direction(args[4])) 
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,958,099 tx:0xa4fbc5c3631095ab5bd1fe388808a9772db97b025bd76b66fe5205f3653bccf3
Swap at block:60,958,099 tx:0xa4fbc5c3631095ab5bd1fe388808a9772db97b025bd76b66fe5205f3653bccf3
Swap at block:60,958,099 tx:0xa4fbc5c3631095ab5bd1fe388808a9772db97b025bd76b66fe5205f3653bccf3
Swap at block:60,958,101 tx:0x99381b2f9b40ea7037b956e197cbf5157c50e2c31489d27f5e2be8edcb5ce5b1
Swap at block:60,958,101 tx:0x99381b2f9b40ea7037b956e197cbf5157c50e2c31489d27f5e2be8edcb5ce5b1
Swap at block:60,958,101 tx:0x5c303192e2e9b395c405ea383292b207803b73d641231309a7a3249d40de7d06
Swap at block:60,958,102 tx:0x3e004e8a72d9013ae549d136dac6516a6dd0002189735053a83304a7ffd88eb0
Swap at block:60,958,102 tx:0x92b47f121fda93a832a1446cef3dde3516c01f3d678050f8702082c46d5d6856
Swap at block:60,958,102 tx:0x12052bc29cd21ab888e96361812e224ccc702ffdc7bface54aaa7220ca30e477
Swap at block:60,958,103 tx:0x066a53292b58419d10ba610cf498d3c986a6c933f04deac3de3665f28be165ac
Swap at block:60,958,103 tx:0x066a53292b58419d10ba

In [10]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3pool,swap,uniswap_v3,0x1c6b7be94aa44ed276ae2ac65d0b4b36b33ffed5,0xa4fbc5c3631095ab5bd1fe388808a9772db97b025bd7...,60958099,1724452189,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv3pool,swap,uniswap_v3,0xc4823bc18d947dc5c75ebdf9b650111c7b7f936e,0xa4fbc5c3631095ab5bd1fe388808a9772db97b025bd7...,60958099,1724452189,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv3pool,swap,uniswap_v3,0xa374094527e1673a86de625aa59517c5de346d32,0xa4fbc5c3631095ab5bd1fe388808a9772db97b025bd7...,60958099,1724452189,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv3pool,swap,uniswap_v3,0xa374094527e1673a86de625aa59517c5de346d32,0x99381b2f9b40ea7037b956e197cbf5157c50e2c31489...,60958101,1724452193,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv3pool,swap,uniswap_v3,0x41e64a5bc929fa8e6a9c8d7e3b81a13b21ff3045,0x99381b2f9b40ea7037b956e197cbf5157c50e2c31489...,60958101,1724452193,{'web3_type': <class 'web3._utils.datatypes.Sw...
5,polygon,uniswapv3pool,swap,uniswap_v3,0xc42bf5cd16d9eb1e892b66bb32a3892dcb7bb75c,0x5c303192e2e9b395c405ea383292b207803b73d64123...,60958101,1724452193,{'web3_type': <class 'web3._utils.datatypes.Sw...
6,polygon,uniswapv3pool,swap,uniswap_v3,0xa236278bec0e0677a48527340cfb567b4e6e9adc,0x3e004e8a72d9013ae549d136dac6516a6dd000218973...,60958102,1724452195,{'web3_type': <class 'web3._utils.datatypes.Sw...
7,polygon,uniswapv3pool,swap,uniswap_v3,0xbb98b3d2b18aef63a3178023a920971cf5f29be4,0x92b47f121fda93a832a1446cef3dde3516c01f3d6780...,60958102,1724452195,{'web3_type': <class 'web3._utils.datatypes.Sw...
8,polygon,uniswapv3pool,swap,uniswap_v3,0x167384319b41f7094e62f7506409eb38079abff8,0x12052bc29cd21ab888e96361812e224ccc702ffdc7bf...,60958102,1724452195,{'web3_type': <class 'web3._utils.datatypes.Sw...
9,polygon,uniswapv3pool,swap,uniswap_v3,0x7b925e617aefd7fb3a93abe3a701135d7a1ba710,0x066a53292b58419d10ba610cf498d3c986a6c933f04d...,60958103,1724452197,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [11]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'swap',
  'platform': 'uniswap_v3',
  'address': '0x1c6b7be94aa44ed276ae2ac65d0b4b36b33ffed5',
  'tx_hash': '0xa4fbc5c3631095ab5bd1fe388808a9772db97b025bd76b66fe5205f3653bccf3',
  'blk_num': 60958099,
  'timestamp': 1724452189,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0x00000000041d945c46E073F0048cEf510D148dEA',
   'token1': '0x00000000041d945c46E073F0048cEf510D148dEA',
   'amount0': 4690376045937217588,
   'amount1': -141598147,
   'sqrt_price_x96': 437225640201427952770548,
   'liquidity': 19792808042338453,
   'tick': -242160}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'swap',
  'platform': 'uniswap_v3',
  'address': '0xc4823bc18d947dc5c75ebdf9b650111c7b7f936e',
  'tx_hash': '0xa4fbc5c3631095ab5bd1fe388808a9772db97b025bd76b66fe5205f3653bccf3',
  'blk_num': 60958099,
  'timestamp': 1724452189,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0x000